# Задание 4

## 1. Декоратор @cached (0.3 балла)

#### Реализуйте класс для хранения результатов выполнения функции

* max_count - максимальное число хранимых результатов. Если число результатов превышает max_count, требуется выбросить первый результат, т. е. в кеше должно храниться не более max_count последних результатов.
* продумайте архитектуру кеша так, чтобы для функций:

<code>
@cached
def f1():
    pass

@cached
def f2():
    pass
</code>    
должны иметь по max_count хранимых последних результатов, и т. д.

<b>P. S.</b>

* Считайте, что функция не имеет состояния (зависит только от передаваемых в нее аргументов).
* Храните данные так, чтобы из функции нельзя напрямую было получить закешированные результаты (только через \_\_closer\_\_).

<b>Рекомендации:</b>

* Для хранения данных используйте OrderedDict.
* Декорируйте wrapper с @functools.wraps(func)

In [1]:
from collections import OrderedDict
import functools
class LruCache(object):
    def __init__(self, max_count):
        self.cache = OrderedDict()
        self.max_count = max_count
    def __getitem__(self, key):
        return self.cache[key]
    def __setitem__(self, key, value):
        self.cache[key] = value

#### Реализуйте декоратор

In [2]:
def cached(max_count=32):
    memory = LruCache(max_count)
    def decore(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            args = tuple(args)
            kwargs_keys = tuple(kwargs.keys())
            kwargs_values = tuple(kwargs.values())
            key = args + kwargs_keys + kwargs_values #create hashable key from arguments
            if key in memory.cache.keys():
                print('from cache') #to see if value was taken from cache
                return memory[key]
            
            else:
                value = func(*args, **kwargs)
                memory[key] = value
                if len(memory.cache) > memory.max_count:
                    memory.cache.popitem(last=False)
                return value
        return wrapper
    return decore      

#### Проверьте использование декоратора

In [3]:
import time
@cached(3)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

@cached(20)
def fact1(n):
    if n < 2:
        return 1
    return fact1(n-1) * n

@cached(10)
def many_args(*args, **kwargs):
    return tuple(args) + tuple(kwargs)

print('call fact(10) -', fact(10), '\n') #now 10, 9, 8 in cache
print('call fact(10), fact(9), fact(8) (all in cache) -', fact(10), fact(9), fact(8), '\n')
print('call fact(5) (not in cache) -', fact(5))
print('\nfunction name is', fact.__name__, '\n')
print('call fact1(5) then fact1(8) (fact1(5) in cache on fact1(8) call) -', fact1(5), fact1(8)) #fact1 has different cache

print('\nmany_args test:')
many_args(1,2,3, a=10, b=20)
many_args(1,2,3, a=10, b=20) #in cache
many_args(1,2,3, b=20, a=10) #in cache, same as many_args(1,2,3, a=10, b=20) -> order of kwargs is not important
start_time = time.time()
end_time = time.time()
print('elapsed time =', end_time - start_time)

call fact(10) - 3628800 

from cache
from cache
from cache
call fact(10), fact(9), fact(8) (all in cache) - 3628800 362880 40320 

call fact(5) (not in cache) - 120

function name is fact 

from cache
call fact1(5) then fact1(8) (fact1(5) in cache on fact1(8) call) - 120 40320

many_args test:
from cache
from cache
elapsed time = 3.123283386230469e-05


Все работает. Разные функции имеют разные кэши. От перестановки мест kwargs кэширование не зависит(чего нет в lru_cahce судя по документации).

#### Сравните свою реализацию с lru_cache из functools

Для этого удалим строку с print в написанном декораторе, чтобы можно было сравнивать время работы.

In [4]:
def cached(max_count=32):
    memory = LruCache(max_count)
    def decore(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            args = tuple(args)
            kwargs_keys = tuple(kwargs.keys())
            kwargs_values = tuple(kwargs.values())
            key = args + kwargs_keys + kwargs_values #create hashable key from arguments
            if key in memory.cache.keys():
                return memory[key]
            
            else:
                value = func(*args, **kwargs)
                memory[key] = value
                if len(memory.cache) > memory.max_count:
                    memory.cache.popitem(last=False)
                return value
        return wrapper
    return decore      

In [5]:
import time
from functools import lru_cache
start_time = time.time()
@lru_cache(10)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

start_time = time.time()
fact(100)
fact(95)
fact(50)
end_time = time.time()
print('lru_cache elapsed time =', end_time - start_time)

start_time = time.time()
@cached(10)
def fact(n):
    if n < 2:
        return 1
    return fact(n-1) * n

start_time = time.time()
fact(100)
fact(95)
fact(50)
end_time = time.time()
print('my realisation elapsed time =', end_time - start_time)

lru_cache elapsed time = 0.0012249946594238281
my realisation elapsed time = 0.0018377304077148438


Время работы моей реализации и встроенной сравнимо. Оно меняется от запуска к запуску, бывает больше одно, бывает другое.

### Дополнительное задание (0.2 балла)

Дополните декоратор @cached так, чтобы не пересчитывать функцию при изменения ее состояния (например, она использовала глобальную переменную)

In [6]:
#WTF?!

## 2. Декоратор @checked (0.3 балла)

Напишите декоратор, который будет вызывать исключение (raise TypeError), если в него переданы аргументы не тех типов.

<b>P. S.</b> Разберитесь с модулем typing.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Чтобы кинуть иключение используйте конструкцию типа:
<code>
if < some_condtion >:
    raise TypeError
</code>

In [7]:
import functools
def checked(*argstypes, **kwargstypes):
    def decore(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            if len(args) > len(argstypes):
                raise TypeError('{}() takes at most {} non-keyword arguments ({} given)' \
                                .format(func.__name__, len(argstypes), len(args)))
            argspairs = []
            for arg, type_ in zip(args, argstypes):
                argspairs.append((arg, type_))
            for k, v in kwargs.items():
                if k not in kwargstypes:
                    raise TypeError("Unexpected keyword argument '%s' for %s()" % (k, func.__name__))
                argspairs.append((v, kwargstypes[k]))
            for param, expected in argspairs:
                if param is not None and not isinstance(param, expected):
                    raise TypeError("Parameter '%s' is not %s" % (param, expected.__name__))
            return func(*args, **kwargs)
        return wrapper
    return decore

#### Проверьте использование декоратора

In [8]:
@checked(str, arg2=int)
def f(arg1, arg2=None):
    pass

f('foo') #works fine
f('foo', arg2=3) #works fine

In [9]:
f('foo', 'bar') #error -> takes at most 1 non-keyword

TypeError: f() takes at most 1 non-keyword arguments (2 given)

In [10]:
f('foo', arg2='bar') #error -> arg2 expected to be int

TypeError: Parameter 'bar' is not int

In [11]:
f(1, arg2='bar') #error -> first arg expected to be str

TypeError: Parameter '1' is not str

## 3. Декоратор @Logger (0.4 балла)

Напишите полноценный logger для вызовов вашей функции. Декоратор должен иметь следующие опции:

* Выбор файла в который будет производиться запись: sys.stdout, sys.stderr, локальный файл (передается путь к файлу, если файла нет, то создать, иначе дописывать в конец).
* Формат записи в логера: "<i>index data time functio_name \*args \**kwargs result</i>"
* Логер должен быть один для всех функций.

<b>Рекомендации:</b>

* Декорируйте wrapper с @functools.wraps(func)
* Создайте отдельный класс Logger для работы с выводом данных вызовов функций в файл.

In [12]:
import sys
import functools
import datetime

#сделаем функцию синглтон, чтобы задекорировать ею класс _Logger
def singleton(cls):
    instance = None
    @functools.wraps(cls)
    def wrapper(*args, **kwargs):
        nonlocal instance
        if instance is None:
            instance = cls(*args, **kwargs)
        return instance
    return wrapper

#сам класс логгер, один на все функции т.к. синглтон
@singleton
class _Logger(object):
    def __init__(self):
        self.index = 0
        
    def log_function_call(self, func, dest, *args, **kwargs):
        now = datetime.datetime.now()
        func_output = func(*args, **kwargs)
        if dest == sys.stderr or dest == sys.stdout:
            print(self.index, str(now), func.__name__, args, kwargs, func_output, file=dest)
        else:
            f = open(dest, 'a')
            f.write('{} {} {} {} {} {}\n'.format(self.index, str(now), func.__name__, \
                                                  args, kwargs, func_output))
            f.close()
        self.index += 1
        return
    
#объявляем еще одну функцию, чтобы можно было вызывать декоратор @Logger с параметрами    
def Logger(dest):
    logger = _Logger()
    def decore(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            return logger.log_function_call(func, dest, *args, **kwargs)
        return wrapper
    return decore

In [13]:
@Logger(sys.stderr)
def test():
    return 'Hello from test'

@Logger(sys.stdout)
def test2():
    return 'Hello from test2'

@Logger('test3.txt')
def test3():
    return 'Hello from test3'
test()
test2()
test3()

1 2018-03-28 19:54:01.447040 test2 () {} Hello from test2


0 2018-03-28 19:54:01.446169 test () {} Hello from test
